In [1]:
import pandas as pd
import ast
import json
import base64
import numpy as np
import os
import cv2, random
np.set_printoptions(suppress=True, precision=4)
from scipy import spatial
import numpy as np
import yaml
import os.path as op
from tqdm import tqdm
import nltk
from nltk.corpus import wordnet
tqdm.pandas()

### Features Detection

In [26]:
sg_tsv = './training_detection/train.feature.tsv'
df_feature_detection = pd.read_csv(sg_tsv,sep='\t',header = None, converters={1:json.loads}, skiprows = 20000, nrows = 10000)
df_feature_detection = df_feature_detection.rename(columns={0: "id", 1: "features"})

In [27]:
sg_tsv = './training_detection/train.label.tsv'
df_label_detection = pd.read_csv(sg_tsv,sep='\t',header = None, converters={1:json.loads}, skiprows = 20000, nrows = 10000)
df_label_detection=df_label_detection.rename(columns={0: "id", 1: "label"})
#df_label_detection

In [28]:
np.all(df_label_detection["id"].values == df_feature_detection["id"].values)

True

In [29]:
full_detection = pd.merge(df_feature_detection, df_label_detection, on="id")
full_detection

,id,features,label
0,4679648220,{'features': 'AAAAAAAAAAAAAAAA8JoNP85JNT4AAAAA...,"[{'class': 'Belt', 'conf': 0.9678944945335388,..."
1,4679736464,{'features': 'AAAAAAAAAAAAAAAAhkdEPwAAAABIoKs+...,"[{'class': 'Face', 'conf': 0.923794686794281, ..."
2,4679761535,{'features': 'WYDnPwAAAAAAAAAAsZG9QHhGVz0AAAAA...,"[{'class': 'Dress', 'conf': 0.9395199418067932..."
3,4679771579,{'features': 'AAAAAFNwhDwAAAAAjzkOQQAAAAB5x4Q/...,"[{'class': 'Train', 'conf': 0.9594755172729492..."
4,4679881654,{'features': 'AAAAAAAAAAAAAAAACVxNPQAAAAB6Y/w9...,"[{'class': 'Belt', 'conf': 0.9516404867172241,..."
...,...,...,...
8995,997338199,{'features': 's9yMPgAAAAAAAAAAWzh9P5uqOEAAAAAA...,"[{'class': 'Hair', 'conf': 0.7632176876068115,..."
8996,997722733,{'features': 'N7IGQElTrj0AAAAAxUgmPkTlSD8AAAAA...,"[{'class': 'Man', 'conf': 0.9461695551872253, ..."
8997,997876722,{'features': 'AAAAAF9gpD8AAAAAubgjPQAAAAAAAAAA...,"[{'class': 'Tomato', 'conf': 0.910000860691070..."
8998,99804383,{'features': 'iGkyPi+yeT0AAAAAMgceQHxPhDwAAAAA...,"[{'class': 'Shirt', 'conf': 0.9361402988433838..."


In [30]:
from tqdm import tqdm
bbox_id = 0
for index, row in tqdm(full_detection.iterrows()):
    list_features = np.split(np.frombuffer(base64.b64decode(row["features"]["features"]), np.float32), row["features"]["num_boxes"])
    for i in range(row["features"]["num_boxes"]):
        row["label"][i]["feature"] = list_features[i]
        row["label"][i]["bbox_id"] = bbox_id
        bbox_id+=1

9000it [01:10, 127.52it/s]


### Features Segmentation

In [31]:
sg_tsv = './features_test1/training/train.feature.tsv'
df_feature_segmentation = pd.read_csv(sg_tsv,sep='\t',header = None, converters={1:json.loads}, skiprows = 20000, nrows = 10000)
df_feature_segmentation = df_feature_segmentation.rename(columns={0: "id", 1: "features"})

In [32]:
sg_tsv = './features_test1/training/train.label.tsv'
df_label_segmentation = pd.read_csv(sg_tsv,sep='\t',header = None, converters={1:json.loads}, skiprows = 20000, nrows = 10000)
df_label_segmentation = df_label_segmentation.rename(columns={0: "id", 1: "label"})
#df_label_segmentation

In [33]:
np.all(df_feature_segmentation["id"].values == df_label_segmentation["id"].values)

True

In [34]:
full_segmentation = pd.merge(df_feature_segmentation, df_label_segmentation, on="id")
full_segmentation

,id,features,label
0,4679648220,{'features': 'V9OhP+JB4UCWO41Ban4SQRZjtEAAAAAA...,"[{'class': 'Bicycle', 'conf': 0.97119373083114..."
1,4679736464,{'features': '4jMaQbnrkEHZc3JAmLC7P0mkbkEAAAAA...,"[{'class': 'Life jacket', 'conf': 0.9988144636..."
2,4679761535,{'features': 'wA33QL8ifkHEDVRAQXEOQRc/UkB0TSFA...,"[{'class': 'Dress', 'conf': 0.975189208984375,..."
3,4679771579,{'features': 'A3zNQMSSyEBsZfU+n5+yQChyskBOZgw/...,"[{'class': 'Skirt', 'conf': 0.9394164085388184..."
4,4679881654,{'features': 'NvofQf5EQUBYbn9B3oalQIritD8Naes/...,"[{'class': 'Flag', 'conf': 0.8809290528297424,..."
...,...,...,...
8995,997338199,{'features': 'aPgEQQAAAAAAAAAAJX5TQAAAAADvJSY+...,"[{'class': 'Curtain', 'conf': 0.80887413024902..."
8996,997722733,{'features': '9pzPP7/XVUEAAAAAnGWUPuE1T0H3ks1A...,"[{'class': 'Sandal', 'conf': 0.717478156089782..."
8997,997876722,{'features': 'mPJwQAAAAAAAAAAAuo71PzYZgkAaSe5A...,"[{'class': 'Apple', 'conf': 0.9743674397468567..."
8998,99804383,{'features': 'SwyYQBGga0A6YDY/AAAAAJCAgj8fi/s/...,"[{'class': 'Signboard', 'conf': 0.706024229526..."


In [35]:
from tqdm import tqdm
bbox_id = 0
for index, row in tqdm(full_segmentation.iterrows()):
    list_features = np.split(np.frombuffer(base64.b64decode(row["features"]["features"]), np.float32), row["features"]["num_boxes"])
    for i in range(row["features"]["num_boxes"]):
        row["label"][i]["feature"] = list_features[i]
        row["label"][i]["bbox_id"] = bbox_id
        bbox_id+=1

9000it [00:29, 303.14it/s]


In [36]:
np.all(full_segmentation["id"].values == full_detection["id"].values)

True

In [37]:
import en_core_web_md
nlp = en_core_web_md.load()

In [38]:
bbox_similarity = 0.997

for index, row in tqdm(full_segmentation.iterrows()):
    for elem in row["label"]:
        similar_region = {"class": "", "score": 0, "feature": [], "conf": 0}
        similar_regions = {}
        similar_regions_features = {}
        similar_features = {"class": "", "score": 0, "feature": [], "conf": 0}
        for elem2 in full_detection.iloc[index]["label"]:

            if 1 - spatial.distance.cosine(elem["rect"], elem2["rect"]) > bbox_similarity:
                #print("similar region class:", elem2["class"])
                similarity = 1 - spatial.distance.cosine(elem["rect"], elem2["rect"])
                sim_features = 1 - spatial.distance.cosine(elem["feature"], elem2["feature"])
                if (sim_features > similar_features["score"]): # 3
                    similar_features["score"] = sim_features
                    similar_features["class"] = elem2["class"]
                    similar_features["feature"] = elem2["feature"]
                    similar_features["conf"] = elem2["conf"]
                if (similarity > similar_region["score"]): # 1
                    similar_region["score"] = similarity
                    similar_region["class"] = elem2["class"]
                    similar_region["feature"] = elem2["feature"]
                    similar_region["conf"] = elem2["conf"]
                if elem2["class"] in similar_regions: # 2
                    similar_regions[elem2["class"]] += 1
                    if similarity > similar_regions_features[elem2["class"]]["score"]: 
                        similar_regions_features[elem2["class"]] = {"score": similarity, "feature": elem2["feature"], "conf": elem2["conf"]}
                else:
                    similar_regions[elem2["class"]] = 0
                    similar_regions_features[elem2["class"]] = {"score": similarity, "feature": elem2["feature"], "conf": elem2["conf"]}
        if similar_region["class"] != "":
            #print("\n======================================================================")
            final_class = elem["class"]
            elem["class"] = elem["class"].lower()
            #print("Classe vecchia: ", elem["class"])
            classe = max(similar_regions, key=similar_regions.get)
            #print("Classe nuova metodo 1: ", classe)
            sim1 = 1 - spatial.distance.cosine(elem["feature"], similar_regions_features[classe]["feature"])
            tokens = nlp(elem["class"].lower() + " " + classe.lower())
            token1, token2 = tokens[0], tokens[1]
            similarity_seg_1 = token1.similarity(token2)
            #print("Classe nuova metodo 2: ", similar_region["class"])
            sim2 = 1 - spatial.distance.cosine(elem["feature"], similar_region["feature"])
            tokens = nlp(elem["class"].lower() + " " + similar_region["class"].lower())
            token1, token2 = tokens[0], tokens[1]
            similarity_seg_2 = token1.similarity(token2)
            #print("Classe nuova metodo 3: ", similar_features["class"])
            tokens = nlp(elem["class"].lower() + " " + similar_features["class"].lower())
            token1, token2 = tokens[0], tokens[1]
            similarity_seg_3 = token1.similarity(token2)

            peso_feat = 0.6
            peso_word = 0.3
            peso_conf = 0.1

            score_1_sim_feat = similarity_seg_1
            score_1_sim_word = sim1
            score_1_conf = similar_regions_features[classe]["conf"]
            score_1_tot = (score_1_sim_feat*peso_feat + score_1_sim_word*peso_word + score_1_conf*peso_conf)

            score_2_sim_feat = similarity_seg_2
            score_2_sim_word = sim2
            score_2_conf = similar_region["conf"]
            score_2_tot = (score_2_sim_feat*peso_feat + score_2_sim_word*peso_word + score_2_conf*peso_conf)

            score_3_sim_feat = similarity_seg_3
            score_3_sim_word = similar_features["score"]
            score_3_conf = similar_features["conf"]
            score_3_tot = (score_3_sim_feat*peso_feat + score_3_sim_word*peso_word + score_3_conf*peso_conf)

            
            if score_1_tot >= score_2_tot and score_1_tot >= score_3_tot:
                final_class = classe
            elif score_2_tot >= score_1_tot and score_2_tot >= score_3_tot:
                final_class = similar_region["class"]
            elif score_2_tot >= score_1_tot and score_3_tot >= score_2_tot:
                final_class = similar_features["class"]
            elem["class"] = final_class
            #print("classe finale: ", elem["class"])
            #print("======================================================================\n")
            
            
            #img = str(full_detection.iloc[index]["id"])+".jpg"
            #im_cv2 = cv2.imread("../datasets/flickr30k_images/validation/"+img)
            #x,y,w,h = tuple(elem["rect"])
            #ROI = im_cv2[int(y):int(h), int(x):int(w)]
            #cv2.imshow(str(elem["class"]),ROI)
            #cv2.waitKey(0) 
            #cv2.destroyAllWindows()

222it [01:57,  2.22it/s]C:\Users\GABRIE~1.FER\AppData\Local\Temp/ipykernel_10580/4069507114.py:42: UserWarning: [W008] Evaluating Token.similarity based on empty vectors.
  similarity_seg_1 = token1.similarity(token2)
C:\Users\GABRIE~1.FER\AppData\Local\Temp/ipykernel_10580/4069507114.py:47: UserWarning: [W008] Evaluating Token.similarity based on empty vectors.
  similarity_seg_2 = token1.similarity(token2)
C:\Users\GABRIE~1.FER\AppData\Local\Temp/ipykernel_10580/4069507114.py:51: UserWarning: [W008] Evaluating Token.similarity based on empty vectors.
  similarity_seg_3 = token1.similarity(token2)
9000it [44:02,  3.41it/s]


### Union

In [39]:
full_not_present = []
for i in tqdm(range(len(full_detection))):    
    not_present = []
    for row_dect in full_detection.iloc[i]["label"]:
        check = False
        for row_seg in full_segmentation.iloc[i]["label"]:
            if 1 - spatial.distance.cosine(row_seg["rect"], row_dect["rect"]) > 0.985:
                check = True
                break
        if not check:
            not_present.append(row_dect)
    full_not_present.append(not_present)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9000/9000 [03:40<00:00, 40.78it/s]


In [40]:
for elem in full_detection["id"].values == full_segmentation["id"].values:
    if not elem: 
        print("noooo")

In [41]:
full_segmentation["not_present"] = full_not_present

In [42]:
full_segmentation["union"] = full_segmentation["label"] + full_segmentation["not_present"]
full_segmentation

,id,features,label,not_present,union
0,4679648220,{'features': 'V9OhP+JB4UCWO41Ban4SQRZjtEAAAAAA...,"[{'class': 'Bicycle', 'conf': 0.97119373083114...","[{'class': 'Belt', 'conf': 0.9678944945335388,...","[{'class': 'Bicycle', 'conf': 0.97119373083114..."
1,4679736464,{'features': '4jMaQbnrkEHZc3JAmLC7P0mkbkEAAAAA...,"[{'class': 'Head', 'conf': 0.9988144636154175,...","[{'class': 'Water', 'conf': 0.6539292335510254...","[{'class': 'Head', 'conf': 0.9988144636154175,..."
2,4679761535,{'features': 'wA33QL8ifkHEDVRAQXEOQRc/UkB0TSFA...,"[{'class': 'Dress', 'conf': 0.975189208984375,...","[{'class': 'Shadow', 'conf': 0.884247660636901...","[{'class': 'Dress', 'conf': 0.975189208984375,..."
3,4679771579,{'features': 'A3zNQMSSyEBsZfU+n5+yQChyskBOZgw/...,"[{'class': 'Pant', 'conf': 0.9394164085388184,...","[{'class': 'Window', 'conf': 0.949151456356048...","[{'class': 'Pant', 'conf': 0.9394164085388184,..."
4,4679881654,{'features': 'NvofQf5EQUBYbn9B3oalQIritD8Naes/...,"[{'class': 'Belt', 'conf': 0.8809290528297424,...","[{'class': 'Tower', 'conf': 0.8458567261695862...","[{'class': 'Belt', 'conf': 0.8809290528297424,..."
...,...,...,...,...,...
8995,997338199,{'features': 'aPgEQQAAAAAAAAAAJX5TQAAAAADvJSY+...,"[{'class': 'Curtain', 'conf': 0.80887413024902...","[{'class': 'Wall', 'conf': 0.7054150104522705,...","[{'class': 'Curtain', 'conf': 0.80887413024902..."
8996,997722733,{'features': '9pzPP7/XVUEAAAAAnGWUPuE1T0H3ks1A...,"[{'class': 'Sandal', 'conf': 0.717478156089782...","[{'class': 'Hair', 'conf': 0.7762690186500549,...","[{'class': 'Sandal', 'conf': 0.717478156089782..."
8997,997876722,{'features': 'mPJwQAAAAAAAAAAAuo71PzYZgkAaSe5A...,"[{'class': 'Apple', 'conf': 0.9743674397468567...","[{'class': 'Pant', 'conf': 0.8092154264450073,...","[{'class': 'Apple', 'conf': 0.9743674397468567..."
8998,99804383,{'features': 'SwyYQBGga0A6YDY/AAAAAJCAgj8fi/s/...,"[{'class': 'Signboard', 'conf': 0.706024229526...","[{'class': 'Shirt', 'conf': 0.9361402988433838...","[{'class': 'Signboard', 'conf': 0.706024229526..."


In [43]:
for index, row in tqdm(full_segmentation.iterrows()):
    if(len(row["label"]) + len(row["not_present"]) != len(row["union"])):
        print("!!!!!!!!!!!!!!!")

9000it [00:00, 10815.84it/s]


In [44]:
def generate_features(x):
    idx, data, num_boxes = x["id"],x["union"],len(x["union"])
    features_arr = []
    for i in range(num_boxes):
        features = data[i]['feature']
        features_arr.append(features.astype(np.float32))
    
    features = np.vstack(tuple(features_arr))
    features = base64.b64encode(features).decode("utf-8")
    return {"features":features, "num_boxes":num_boxes}

In [45]:
def generate_labels(x):
    data = x["union"]
    res = [{"class":el['class'].capitalize(),"conf":el['conf'], "rect": el['rect']} for el in data] 
    return res

In [46]:
full_segmentation['feature_union'] = full_segmentation.progress_apply(generate_features,axis=1)
full_segmentation['feature_union'] = full_segmentation['feature_union'].progress_apply(json.dumps)

full_segmentation['label_union'] = full_segmentation.progress_apply(generate_labels,axis=1)
full_segmentation['label_union'] = full_segmentation['label_union'].progress_apply(json.dumps)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9000/9000 [00:01<00:00, 5460.43it/s]


In [47]:
from features_detection.maskrcnn_benchmark.structures.tsv_file_ops import tsv_reader, tsv_writer

OUTPUT_DIR = './training_union_class/'
LABEL_FILE = os.path.join(OUTPUT_DIR,'train3.label.tsv')
FEATURE_FILE = os.path.join(OUTPUT_DIR,'train3.feature.tsv')
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)
    print(f"path to {OUTPUT_DIR} created")

In [48]:
tsv_writer(full_segmentation[["id", "feature_union"]].values.tolist(), FEATURE_FILE)

In [49]:
tsv_writer(full_segmentation[["id", "label_union"]].values.tolist(),LABEL_FILE)

Union

In [26]:
sg_tsv = './training_union_class/train1.feature.tsv'
df_feature1 = pd.read_csv(sg_tsv,sep='\t',header = None, converters={1:json.loads})
df_feature1 = df_feature1.rename(columns={0: "id", 1: "features"})

In [3]:
sg_tsv = './training_union_class/train2.feature.tsv'
df_feature2 = pd.read_csv(sg_tsv,sep='\t',header = None, converters={1:json.loads})
df_feature2 = df_feature2.rename(columns={0: "id", 1: "features"})

In [12]:
sg_tsv = './training_union_class/train3.feature.tsv'
df_feature3 = pd.read_csv(sg_tsv,sep='\t',header = None, converters={1:json.loads})
df_feature3 = df_feature3.rename(columns={0: "id", 1: "features"})

In [13]:
df_feature = pd.concat([df_feature1, df_feature2, df_feature3], ignore_index=True)
df_feature

,id,features
0,1000092795,{'features': 'ki89QAAAAAAAAAAAFvs8P3NUhz/ght1A...
1,10002456,{'features': 'Ta8CQRDNxD81+ZRAPhfGQOb8wkCgvb5A...
2,1000268201,{'features': 'fKqDQQAAAAA8P44/WHkaQfywkEAAAAAA...
3,1000344755,{'features': 'u24FQFOHokAAAAAACkI9QBn70T8kwudA...
4,1000366164,{'features': 'lJYiQV6dlT9WZDlAXJmOQL94t0AAshc8...
...,...,...
28995,997338199,{'features': 'aPgEQQAAAAAAAAAAJX5TQAAAAADvJSY+...
28996,997722733,{'features': '9pzPP7/XVUEAAAAAnGWUPuE1T0H3ks1A...
28997,997876722,{'features': 'mPJwQAAAAAAAAAAAuo71PzYZgkAaSe5A...
28998,99804383,{'features': 'SwyYQBGga0A6YDY/AAAAAJCAgj8fi/s/...


In [27]:
sg_tsv = './training_union_class/train1.label.tsv'
df_label1 = pd.read_csv(sg_tsv,sep='\t',header = None, converters={1:json.loads})
df_label1 = df_label1.rename(columns={0: "id", 1: "label"})

In [6]:
sg_tsv = './training_union_class/train2.label.tsv'
df_label2 = pd.read_csv(sg_tsv,sep='\t',header = None, converters={1:json.loads})
df_label2 = df_label2.rename(columns={0: "id", 1: "label"})

In [7]:
sg_tsv = './training_union_class/train3.label.tsv'
df_label3 = pd.read_csv(sg_tsv,sep='\t',header = None, converters={1:json.loads})
df_label3 = df_label3.rename(columns={0: "id", 1: "label"})

In [10]:
df_label = pd.concat([df_label1, df_label2, df_label3], ignore_index=True)
df_label

,id,label
0,1000092795,"[{'class': 'Hand', 'conf': 0.5350186228752136,..."
1,10002456,"[{'class': 'Traffic light', 'conf': 0.51291126..."
2,1000268201,"[{'class': 'Dress', 'conf': 0.9662193059921265..."
3,1000344755,"[{'class': 'Hand', 'conf': 0.7075430750846863,..."
4,1000366164,"[{'class': 'Pot', 'conf': 0.8574621677398682, ..."
...,...,...
28995,997338199,"[{'class': 'Curtain', 'conf': 0.80887413024902..."
28996,997722733,"[{'class': 'Sandal', 'conf': 0.717478156089782..."
28997,997876722,"[{'class': 'Apple', 'conf': 0.9743674397468567..."
28998,99804383,"[{'class': 'Signboard', 'conf': 0.706024229526..."


In [14]:
np.all(df_label["id"].values == df_feature["id"].values)

True

In [15]:
sg_tsv = './training_union_class/train.hw.tsv'
df_hw = pd.read_csv(sg_tsv,sep='\t',header = None, converters={1:json.loads})
df_hw = df_hw.rename(columns={0: "id", 1: "dim"})
df_hw

,id,dim
0,1000092795,"[{'height': 500, 'width': 333}]"
1,10002456,"[{'height': 374, 'width': 500}]"
2,1000268201,"[{'height': 500, 'width': 375}]"
3,1000344755,"[{'height': 333, 'width': 500}]"
4,1000366164,"[{'height': 375, 'width': 500}]"
...,...,...
28995,997338199,"[{'height': 400, 'width': 500}]"
28996,997722733,"[{'height': 500, 'width': 333}]"
28997,997876722,"[{'height': 500, 'width': 333}]"
28998,99804383,"[{'height': 500, 'width': 375}]"


In [17]:
np.all(df_hw["id"].values == df_feature["id"].values)

True

In [18]:
np.all(df_hw["id"].values == df_label["id"].values)

True

In [19]:
from features_detection.maskrcnn_benchmark.structures.tsv_file_ops import tsv_reader, tsv_writer

OUTPUT_DIR = './training_union_class/'
LABEL_FILE = os.path.join(OUTPUT_DIR,'train.label.tsv')
FEATURE_FILE = os.path.join(OUTPUT_DIR,'train.feature.tsv')
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)
    print(f"path to {OUTPUT_DIR} created")

In [21]:
df_feature['features'] = df_feature['features'].progress_apply(json.dumps)

100%|███████████████████████████████████████████████████████████████████████████| 29000/29000 [04:39<00:00, 103.73it/s]


In [20]:
df_label['label'] = df_label['label'].progress_apply(json.dumps)

100%|██████████████████████████████████████████████████████████████████████████| 29000/29000 [00:06<00:00, 4528.42it/s]


In [22]:
tsv_writer(df_feature[["id", "features"]].values.tolist(), FEATURE_FILE)

In [23]:
tsv_writer(df_label[["id", "label"]].values.tolist(), LABEL_FILE)